In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch
import torchaudio as ta
import torchaudio.functional as taf
import torchaudio.transforms as tat
from torchvision import transforms

print(torch.__version__)        
print(ta.__version__)

import matplotlib
import matplotlib.pyplot as plt
from IPython.display import Audio, display

import pandas as pd
import os
import pprint

from typing import *
import itertools
from collections import Counter

import numpy as np
from datetime import datetime
from lark.config import Config
from lark.learner import Learner
from lark.ops import Sig2Spec, MixedSig2Spec

1.8.1
0.8.0a0+e4e171a


In [3]:
import torch
# get list of models
torch.hub.list('zhanghang1989/ResNeSt', force_reload=True)

Downloading: "https://github.com/zhanghang1989/ResNeSt/archive/master.zip" to /home/koen/.cache/torch/hub/master.zip


['resnest101',
 'resnest200',
 'resnest269',
 'resnest50',
 'resnest50_fast_1s1x64d',
 'resnest50_fast_1s2x40d',
 'resnest50_fast_1s4x24d',
 'resnest50_fast_2s1x64d',
 'resnest50_fast_2s2x40d',
 'resnest50_fast_4s1x64d',
 'resnest50_fast_4s2x40d']

In [4]:
cfg = Config(
#     n_mels = 128,
#     f_min = 20,
#     f_max = 16000,
#     n_fft = 2048,
#     hop_length = 512,
    
    noise_nsr_dbs = [35, 30, 20, 10],
    sites=None, 
    use_neptune=True, 
    n_epochs=15, 
    bs=32, 
    lr=1e-4, 
    model='resnest50',
    scheduler='torch.optim.lr_scheduler.OneCycleLR'
)

In [5]:
cfg.as_dict()

{'sites': None,
 'data_dir': 'data/birdclef-2021',
 'checkpoint_dir': 'checkpoints',
 'bs': 32,
 'n_workers': 12,
 'duration': 5,
 'use_noise': True,
 'noise_nsr_dbs': [35, 30, 20, 10],
 'noise_dir': 'data/noise/BirdVox-DCASE-20k/wav-32k',
 'use_overlays': True,
 'max_overlays': 5,
 'overlay_weights': [0.71986223,
  0.21010333,
  0.06314581,
  0.00574053,
  0.00114811],
 'overlay_snr_dbs': [20, 10, 3],
 'use_neptune': True,
 'sr': 32000,
 'n_frames': 160000,
 'n_fft': 512,
 'window_length': 512,
 'n_mels': 64,
 'hop_length': 312,
 'f_min': 150,
 'f_max': 15000,
 'lr': 0.0001,
 'n_epochs': 15,
 'model': 'resnest50',
 'optimizer': 'torch.optim.Adam',
 'loss_fn': 'torch.nn.BCEWithLogitsLoss',
 'scheduler': 'torch.optim.lr_scheduler.OneCycleLR',
 'labels': ['acafly',
  'acowoo',
  'aldfly',
  'ameavo',
  'amecro',
  'amegfi',
  'amekes',
  'amepip',
  'amered',
  'amerob',
  'amewig',
  'amtspa',
  'andsol1',
  'annhum',
  'astfly',
  'azaspi1',
  'babwar',
  'baleag',
  'balori',
  'banan

In [6]:
prep = MixedSig2Spec(cfg)

main_model = torch.hub.load('zhanghang1989/ResNeSt', 'resnest50', pretrained=True)

posp = torch.nn.Linear(in_features=2048, 
                       out_features=len(cfg.labels), bias=True)

main_model.fc = posp

model = torch.nn.Sequential(prep, main_model)

model = model.cuda()


Using cache found in /home/koen/.cache/torch/hub/zhanghang1989_ResNeSt_master


In [7]:
lrn = Learner("resnest50-full-training", cfg, model)

In [8]:
lrn.learn()

psutil is not installed. You will not be able to abort this experiment from the UI.
psutil is not installed. Hardware metrics will not be collected.


https://ui.neptune.ai/botkop/lark/e/LAR-65


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/4764 [00:00<?, ?it/s]

skipping file data/birdclef-2021/train_short_audio.wav/cogdov/XC579430.wav offset 943031: invalid signal
skipping file data/birdclef-2021/train_short_audio.wav/babwar/XC137588.wav offset 165386: invalid signal
skipping file data/birdclef-2021/train_short_audio.wav/bkmtou1/XC518213.wav offset 54628: invalid signal
skipping file data/birdclef-2021/train_short_audio.wav/cogdov/XC579430.wav offset 638457: invalid signal
skipping file data/birdclef-2021/train_short_audio.wav/cogdov/XC579430.wav offset 979807: invalid signal
skipping file data/birdclef-2021/train_short_audio.wav/cogdov/XC579430.wav offset 46297: invalid signal
skipping file data/birdclef-2021/train_short_audio.wav/trokin/XC604129.wav offset 1121319: invalid signal
skipping file data/birdclef-2021/train_short_audio.wav/cogdov/XC579430.wav offset 485360: invalid signal
skipping file data/birdclef-2021/train_short_audio.wav/cogdov/XC579430.wav offset 473634: invalid signal
skipping file data/birdclef-2021/train_short_audio.wav/

...

2021-05-04 01:03:27 epoch:  14 train loss: 0.012510 train f1: 0.482933 valid loss: 0.008920 valid f1: 0.065523


  0%|          | 0/4764 [00:00<?, ?it/s]

skipping file data/birdclef-2021/train_short_audio.wav/babwar/XC137588.wav offset 150894: invalid signal
skipping file data/birdclef-2021/train_short_audio.wav/babwar/XC137588.wav offset 182641: invalid signal
skipping file data/birdclef-2021/train_short_audio.wav/babwar/XC137588.wav offset 179925: invalid signal
skipping file data/birdclef-2021/train_short_audio.wav/babwar/XC137588.wav offset 149055: invalid signal
skipping file data/birdclef-2021/train_short_audio.wav/babwar/XC137588.wav offset 150534: invalid signal
skipping file data/birdclef-2021/train_short_audio.wav/babwar/XC137588.wav offset 173158: invalid signal
skipping file data/birdclef-2021/train_short_audio.wav/babwar/XC137588.wav offset 165545: invalid signal
skipping file data/birdclef-2021/train_short_audio.wav/babwar/XC137588.wav offset 153924: invalid signal


  0%|          | 0/75 [00:00<?, ?it/s]

2021-05-04 01:34:38 epoch:  15 train loss: 0.012460 train f1: 0.485824 valid loss: 0.009065 valid f1: 0.058960


In [76]:
lrn.scheduler.state_dict()

{'total_steps': 5040,
 '_schedule_phases': [{'end_step': 1511.0,
   'start_lr': 'initial_lr',
   'end_lr': 'max_lr',
   'start_momentum': 'max_momentum',
   'end_momentum': 'base_momentum'},
  {'end_step': 5039,
   'start_lr': 'max_lr',
   'end_lr': 'min_lr',
   'start_momentum': 'base_momentum',
   'end_momentum': 'max_momentum'}],
 'anneal_func': <bound method OneCycleLR._annealing_cos of <torch.optim.lr_scheduler.OneCycleLR object at 0x7fbe204b0d00>>,
 'cycle_momentum': True,
 'use_beta1': True,
 'base_lrs': [4e-05],
 'last_epoch': 5040,
 '_step_count': 5041,
 'verbose': False,
 '_get_lr_called_within_step': False,
 '_last_lr': [4.198235228265301e-09]}

In [10]:
lrn.name = f"{lrn.name}-latest"
lrn.name

'resnest50-full-training-20210503-174656-latest'

In [11]:
lrn.save_checkpoint(50, 0.009065, 0.058960)

In [12]:
lrn.evaluate()

  0%|          | 0/75 [00:00<?, ?it/s]

,thresh,tp,tn,fp,fn,f1
0,0.0,1183,0,951617,0,0.002480
1,0.1,137,949593,2024,1046,0.081938
2,0.2,90,950880,737,1093,0.089552
3,0.3,68,951284,333,1115,0.085859
4,0.4,53,951466,151,1130,0.076424
5,0.5,38,951549,68,1145,0.058960
6,0.6,27,951578,39,1156,0.043235
7,0.7,22,951596,21,1161,0.035889
8,0.8,15,951609,8,1168,0.024876
9,0.9,5,951614,3,1178,0.008396


In [14]:
lrn.name = 'resnest50-full-training-20210503-174656'
lrn.load_checkpoint()

{'epoch': 4,
 'valid_loss': 0.008272748069527248,
 'valid_score': 0.046985119581222534}

In [15]:
lrn.evaluate()

  0%|          | 0/75 [00:00<?, ?it/s]

,thresh,tp,tn,fp,fn,f1
0,0.0,1183,0,951617,0,0.002480
1,0.1,124,950373,1244,1059,0.097217
2,0.2,69,951273,344,1114,0.086466
3,0.3,46,951445,172,1137,0.065667
4,0.4,37,951513,104,1146,0.055891
5,0.5,30,951553,64,1153,0.046985
6,0.6,22,951576,41,1161,0.035313
7,0.7,13,951592,25,1170,0.021294
8,0.8,9,951603,14,1174,0.014925
9,0.9,6,951610,7,1177,0.010033


In [53]:
for param in model.parameters():
    param.requires_grad = True

In [54]:
lrn.name = 'resnest50-mixed-spec-unfrozen-20210502-144336'

In [55]:
lrn.learn()

psutil is not installed. You will not be able to abort this experiment from the UI.
psutil is not installed. Hardware metrics will not be collected.


https://ui.neptune.ai/botkop/lark/e/LAR-52


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/336 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

...

  0%|          | 0/336 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

2021-05-02 17:03:15 epoch:  30 train loss: 0.115589 train f1: 0.549723 valid loss: 0.123151 valid f1: 0.216257


  0%|          | 0/336 [00:00<?, ?it/s]

KeyboardInterrupt: 